# POPULATION ANALYSIS WITH ZONAL STATISTICS

This code (mentorship legacy project) explores the use of calculating and visualising the population of districts in Madagascar using various python libraries. It builds on all the various libraries and data visualisation tools learnt during the mentorship programme and provides a step by step guide on how to calculate population of an area using zonal stats and geopandas.

**Workflow**
1) Preprocessing: Source data for regions and health facilities as well as population density data for Madagascar
2) Analysis: Computation of population per region using zonal statistics    
3) Result: Visualize results as choropleth maps.

**Datasets used**
1) A Raster shapefile from Worldpop -  https://data.humdata.org/dataset/worldpop-madagascar
2) Health facility data from HDX - https://data.humdata.org/dataset/madagascar-healthsites
3) Madagascar Boundary Data - https://data.humdata.org/dataset/madagascar-administrative-boundary-shapefiles-level-1-4

**Data Visualisation & Output**

Two libraries for data visualisation were used in this code; Matplotlib and Folium. In teh end it was evident that Folium provides more functionality and provides more interactive maps. With the use of popups and a choropleth map with a legend, users can tell the number of people per region and the type of health facilities per region. 

## Analysis

To begin, all the packages or libraries needed to run the code need to be imported into the notebook. 
NB: To be able to import a package, it must already be installed in anaconda 

In [ ]:
#Load all required packages

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
from rasterstats import zonal_stats


import pandas as pd
from shapely.geometry import Polygon
from shapely.geometry import Point
from pyproj import crs
import folium as fol
from folium.plugins import MarkerCluster
from folium.features import Choropleth
import fiona as fn
import numpy
import matplotlib.pyplot as plt
import json
from folium import IFrame

print("All libraries loaded...")

Load all the datasets required as variables. This helps to use the same dataset in different parts of your code

In [ ]:
#Create variables of the files needed to compute zonal statistics

print ("Loading shapefiles.....")
raster= "data_mentorship-20230223T152806Z-001/data_mentorship/mdg_ppp_2020_1km_Aggregated_UNadj.tif"
regions_as_gdf = gpd.read_file("data_mentorship-20230223T152806Z-001/data_mentorship/mdg_adm_bngrc_ocha_20181031_shp/mdg_admbnda_adm1_BNGRC_OCHA_20181031.shp")
health_facilities = gpd.GeoDataFrame.from_file('data_mentorship-20230223T152806Z-001/data_mentorship/madagascar-shapefiles/shapefiles/healthsites.shp')
print ("All shapefiles loaded!")

In [ ]:
#Confirm that ypu have the correct file
regions_as_gdf.head(5)

Calculate Zonal statistics. This will provid ethe sum of people (population) living within the polygon specified. 
In our case, regions in Madagascar. 

In [ ]:
#Calculate Zonal Statistics

regions_as_gdf['sum'] = pd.DataFrame(
    zonal_stats(
        vectors= regions_as_gdf, 
        raster=raster, 
        stats='sum'
    )
)['sum']

In [ ]:
regions_as_gdf.head(5)

#Notice that, a sum field has now been added to the regions geodataframe

Rename the sum field to Population

In [ ]:
regions_as_gdf = regions_as_gdf.rename({'sum': 'Population'}, axis = 'columns')

In [ ]:
#Confirm that the name of the population field has changed from sum to Population

regions_as_gdf.columns

In [ ]:
#Confirm the number of regions

len(regions_as_gdf)

## Visualise the region polygons using matplotlib and folium

**Visualise the regions with matplotlib**

In [ ]:
#Plot or view the regions using matplotlib

#regions_as_gdf.boundary.plot()

print("Styling the map")
fig, ax = plt.subplots(figsize = (12,12))
regions_as_gdf.plot(ax=ax, color="#F0EAD7", edgecolor = "#828282", linewidth =0.25, legend = True)
health_facilities.plot(ax=ax, color="red", legend = True)

#ax.axis('off')

#Add a title to the map
ax.set_title("Population per Region in Madagascar", fontsize = 20, fontweight = "bold", fontname = "Times New Roman")

#Display map
plt.show()

## Visualise the regions using folium

In [ ]:
#convert to Geojson
boundary = regions_as_gdf

#specify the coordinate system and convert to geojson
regions_json = boundary.to_crs(epsg = '4326').to_json()

pop_map = fol.Map(location = [-18.334123, 47.678548], zoom_start = 5)

#Add different basemaps
fol.TileLayer('Stamen Terrain').add_to(pop_map)
fol.TileLayer('cartodbdark_matter').add_to(pop_map)
fol.TileLayer('Stamen water color').add_to(pop_map)

polygons = fol.features.GeoJson(regions_json)
pop_map.add_child(polygons)

fol.LayerControl().add_to(pop_map)
pop_map

In [ ]:
#Add health facilities in madagascar to the map

#convert to Geojson
boundary = regions_as_gdf

#specify the coordinate system and convert to geojson
regions_json = boundary.to_crs(epsg = '4326').to_json()
health_facilities_json = health_facilities.to_crs(epsg = '4326').to_json()

pop_map = fol.Map(location = [-18.334123, 47.678548], zoom_start = 5)

fol.TileLayer('Stamen Terrain').add_to(pop_map)
fol.TileLayer('cartodbdark_matter').add_to(pop_map)
fol.TileLayer('Stamen water color').add_to(pop_map)

polygons = fol.features.GeoJson(regions_json)
points = fol.features.GeoJson(health_facilities_json)
pop_map.add_child(polygons)
pop_map.add_child(points)

fol.LayerControl().add_to(pop_map)
pop_map

In [ ]:
health_facilities .head(1)

Create Popups to show attributes and make the map interactive

In [ ]:

#Create popups
pop_map = fol.Map(location = [-18.334123, 47.678548], zoom_start = 5)

fol.TileLayer('Stamen Terrain').add_to(pop_map)
fol.TileLayer('cartodbdark_matter').add_to(pop_map)
fol.TileLayer('Stamen water color').add_to(pop_map)

#polygons = fol.features.GeoJson(regions_json)
#pop_map.add_child(polygons)

pop_map

## Make a chloropleth map

In [ ]:
#Make a choropleth map and add it to the pop_map
fol.Choropleth(
    geo_data=regions_json,
    name="choropleth",
    data=regions_as_gdf,
    columns=["ADM1_EN", "Population"],
    key_on="feature.properties.ADM1_EN",
    fill_color="YlGn",
    fill_opacity=1,
    line_opacity=0.3,
    highlight = True,
    legend_name="Population Per Region",
    reset = True
).add_to(pop_map)

pop_map

In [ ]:
#create a marker for each health facility and change the symbol to a circle
for index, row in health_facilities.iterrows():
    Facility_type = row['amenity']
    location = (row['lat'], row['long'])
    html =''' Location: {}<br>
    Facility type: {}
    '''.format(location, Facility_type) 

    iframe = fol.IFrame(html,
                       width=300,
                       height=100)

    popup = fol.Popup(iframe,
                     max_width=300, max_height = 300)
    #row_values = row[1]
    marker = fol.Circle(radius = 4, location = location, popup = popup)
    marker.add_to(pop_map)
    
fol.LayerControl().add_to(pop_map)

pop_map

## Output

Save the population map as an html file. Specify a directory and the name of the output map.

In [ ]:
pop_map.save(os.path.join('data_mentorship-20230223T152806Z-001/data_mentorship', 'population_in_madagascar.html'))
